In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
import os

In [2]:
def find_csv_files(folder_path):
    csv_files = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.csv'):
                csv_files.append(os.path.join(root, file))
    return csv_files

In [3]:
def show(df, limit=10):
    return df.toPandas().head(limit)

In [14]:
def union_datasets(df_list, df=None):
    if df == None:
        df = df_list[-1]
    else:
        df = df.unionByName(df_list[0], allowMissingColumns=True)
    
    df_list.pop()

    if df_list:
        return union_datasets(df_list, df)
    else:
        return df   

In [4]:
folder = r"C:\Development\ultimateNakMuay\data\clean"
files = find_csv_files(folder)

In [6]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [11]:
dataframes = {
    k.rsplit("\\", maxsplit=1)[-1].replace(".csv", ""): spark.read.csv(k, header=True) for k in files
}

In [12]:
dataframes

{'bellator': DataFrame[event_num: string, event: string, date: string, venue: string, attendance: string, city: string, state: string, country: string, weight_class: string, winner: string, loser: string, method: string, round: string, time: string, fight_card: string, event_name: string, link: string],
 'glory': DataFrame[event_num: string, event: string, date: string, venue: string, city: string, state: string, country: string, attendance: string, weight_class: string, winner: string, loser: string, method: string, round: string, time: string, notes: string, fight_card: string, event_name: string, link: string],
 'onefc': DataFrame[event_num: string, event: string, date: string, venue: string, city: string, state: string, country: string, attendance: string, weight_class: string, winner: string, loser: string, method: string, round: string, time: string, notes: string, fight_card: string, event_name: string],
 'ufc': DataFrame[event_num: string, event: string, date: string, venue: st

In [22]:
for org, df in dataframes.items():
    dataframes[org] = df.withColumn("organization", F.lit(org))

In [24]:
df = union_datasets(list(dataframes.values()))

In [25]:
show(df)

,event_num,event,date,venue,city,state,country,attendance,weight_class,winner,loser,method,round,time,fight_card,organization,event_name,link
0,653,UFC on ESPN: Strickland vs. Magomedov,2023-07-01,UFC Apex,Las Vegas,Nevada,US,None,Women's Flyweight,Ariane Lipski,Melissa Gatto,"Decision (split) (28–29, 30–27, 29–28)",3.0,300.0,Main card,ufc,None,None
1,653,UFC on ESPN: Strickland vs. Magomedov,2023-07-01,UFC Apex,Las Vegas,Nevada,US,None,Lightweight,Elves Brenner,Guram Kutateladze,TKO (punches),3.0,197.0,Preliminary card (ESPN / ESPN+),ufc,None,None
2,653,UFC on ESPN: Strickland vs. Magomedov,2023-07-01,UFC Apex,Las Vegas,Nevada,US,None,Lightweight,Benoît Saint-Denis,Ismael Bonfim,Submission (rear-naked choke),1.0,288.0,Main card,ufc,None,None
3,653,UFC on ESPN: Strickland vs. Magomedov,2023-07-01,UFC Apex,Las Vegas,Nevada,US,None,Welterweight,Rinat Fakhretdinov,Kevin Lee,Technical Submission (guillotine choke),1.0,55.0,Preliminary card (ESPN / ESPN+),ufc,None,None
4,653,UFC on ESPN: Strickland vs. Magomedov,2023-07-01,UFC Apex,Las Vegas,Nevada,US,None,Featherweight,Joanderson Brito,Westin Wilson,KO (punches),1.0,174.0,Preliminary card (ESPN / ESPN+),ufc,None,None
5,653,UFC on ESPN: Strickland vs. Magomedov,2023-07-01,UFC Apex,Las Vegas,Nevada,US,None,Lightweight,Grant Dawson,Damir Ismagulov,"Decision (unanimous) (30–26, 30–27, 30–27)",3.0,300.0,Main card,ufc,None,None
6,653,UFC on ESPN: Strickland vs. Magomedov,2023-07-01,UFC Apex,Las Vegas,Nevada,US,None,Women's Flyweight,Luana Carolina,Ivana Petrović,"Decision (unanimous) (29–28, 29–28, 29–28)",3.0,300.0,Preliminary card (ESPN / ESPN+),ufc,None,None
7,653,UFC on ESPN: Strickland vs. Magomedov,2023-07-01,UFC Apex,Las Vegas,Nevada,US,None,Heavyweight,Alexander Romanov,Blagoy Ivanov,"Decision (unanimous) (30–27, 30–27, 30–27)",3.0,300.0,Preliminary card (ESPN / ESPN+),ufc,None,None
8,653,UFC on ESPN: Strickland vs. Magomedov,2023-07-01,UFC Apex,Las Vegas,Nevada,US,None,Middleweight,Nursulton Ruziboev,Brunno Ferreira,KO (punches),1.0,77.0,Main card,ufc,None,None
9,653,UFC on ESPN: Strickland vs. Magomedov,2023-07-01,UFC Apex,Las Vegas,Nevada,US,None,Women's Featherweight,Karol Rosa,Yana Santos,"Decision (split) (28–29, 29–28, 29–28)",3.0,300.0,Preliminary card (ESPN / ESPN+),ufc,None,None
